In [1]:
import numpy as np
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import time
from pyDOE import lhs
import matplotlib.pyplot as plt
import matplotlib.ticker
import math
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': print(torch.cuda.get_device_name()) 

cpu


# Data Prep

Training and Testing data is prepared from the solution file

In [2]:
x_1 = np.linspace(-1,1,256)
x_2 = np.linspace(1,-1,256)
x_3 = np.linspace(-1,1,256)

X, Y,Z = np.meshgrid(x_1,x_2,x_3) 

# Test Data

We prepare the test data to compare against the solution produced by the WAN.

In [3]:
X_v_test = np.hstack((X.flatten(order='F')[:,None], Y.flatten(order='F')[:,None], Z.flatten(order='F')[:,None]))

lb = np.array([-1, -1, -1]) #lower bound
ub = np.array([1, 1, 1])  #upper bound

vsol = 1/(4 * np.pi*np.sqrt(X**2 + Y**2+Z**2))+np.sin(X*Y+Z)

v_true = vsol.flatten('F')[:,None] 

# Training Data

In [4]:
def trainingdata(N_v,N_f):
    
    leftedge_x = np.hstack((X[0,:].flatten('a')[:,None], Y[0,:].flatten('a')[:,None], Z[0,:].flatten('a')[:,None]))
    leftedge_v = vsol[0,:].flatten('a')[:,None]
    
    rightedge_x = np.hstack((X[-1,:].flatten('a')[:,None], Y[-1,:].flatten('a')[:,None], Z[-1,:].flatten('a')[:,None]))
    rightedge_v = vsol[-1,:].flatten('a')[:,None]
    
    backedge_x = np.hstack((X[:,0].flatten('a')[:,None], Y[:,0].flatten('a')[:,None], Z[:,0].flatten('a')[:,None]))
    backedge_v = vsol[:,0].flatten('a')[:,None]
    
    frontedge_x = np.hstack((X[:,-1].flatten('a')[:,None], Y[:,-1].flatten('a')[:,None], Z[:,-1].flatten('a')[:,None]))
    frontedge_v = vsol[:,-1].flatten('a')[:,None]
    
    bottomedge_x = np.hstack((X[:,:,0].flatten('a')[:,None], Y[:,:,0].flatten('a')[:,None], Z[:,:,0].flatten('a')[:,None]))
    bottomedge_v = vsol[:,:,0].flatten('a')[:,None]
    
    topedge_x = np.hstack((X[:,:,-1].flatten('a')[:,None], Y[:,:,-1].flatten('a')[:,None], Z[:,:,-1].flatten('a')[:,None]))
    topedge_v = vsol[:,:,-1].flatten('a')[:,None]
    
    all_X_v_train = np.vstack([leftedge_x, rightedge_x, backedge_x, frontedge_x, bottomedge_x, topedge_x])
    all_v_train = np.vstack([leftedge_v, rightedge_v, backedge_v, frontedge_v, bottomedge_v, topedge_v])
     
    #choose random N_v points for training
    idx = np.random.choice(all_X_v_train.shape[0], N_v, replace=False) 
    
    X_v_train = all_X_v_train[idx[0:N_v], :] #choose indices from  set 'idx' (x,t)
    v_train = all_v_train[idx[0:N_v],:]      #choose corresponding v
    
    '''Collocation Points'''

    # N_f sets of tuples(x,t)
    X_f = lb + (ub-lb)*lhs(3,N_f)
    
    return X_f, X_v_train, v_train 

# WAN

Creating sequential layers using the class
tf.Module

In [5]:
class Sequentialmodel0(nn.Module):
    
    def __init__(self,layers0):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
    
        'Initialise neural network as a nn.MSELosslist using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers0[i], layers0[i+1]) for i in range(len(layers0)-1)])
        
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers0)-1):
            
            # weights from a normal distribution with 
            # Recommended gain value for tanh = 5/3?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            

    def forward(self,x):
        
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
                        
        for i in range(len(layers0)-2):
            
            z = self.linears[i](a)
                        
            a = self.activation(z)
            
        a = self.linears[-1](a)
        
        return a
    
    def loss(self,x_to_train_f):

        x_1_f = x_to_train_f[:,[0]]
        x_2_f = x_to_train_f[:,[1]]
        x_3_f = x_to_train_f[:,[2]]
                        
        g = x_to_train_f.clone()
                        
        g.requires_grad = True
        
        uu = self.forward(g)

        uu_x = autograd.grad(uu,g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
                                                            
        uu_x_1 = uu_x[:,[0]]
        
        uu_x_2 = uu_x[:,[1]]
        
        uu_x_3 = uu_x[:,[2]]
                        
        k = x_1_f**2 + x_2_f**2 + x_3_f**2 + 1

        gg = (x_1_f**2 + x_2_f**2 + x_3_f**2 + 1)*(x_1_f**2 + x_2_f**2 +1)*np.sin(x_1_f*x_2_f+x_3_f)-(4*x_1_f*x_2_f+2*x_3_f)*np.cos(x_1_f*x_2_f+x_3_f)+1/(2*np.pi*np.sqrt(x_1_f**2 + x_2_f**2 + x_3_f**2))
        f = torch.log(torch.square(torch.mean(k * (uu_x_1*v_x_1 + uu_x_2*v_x_2+ uu_x_3*v_x_3))- 1e5-torch.mean(gg*uu)))-torch.log(torch.mean(torch.square(uu)))
        
        f = -f

        loss = torch.mean(f)

        return loss
     
    'callable for optimizer'                                       
    def closure(self):
        
        optimizer.zero_grad()
        
        loss_val = self.loss(X_f_train)
        
        #error_vec, _ = WAN.test()
        
        #print(loss_val,error_vec)
        
        loss_val.backward(retain_graph=True)

        return loss_val        
    
    def test(self):
                
        v_pred = self.forward(X_v_test_tensor)
        
        error_vec = torch.linalg.norm((vt-v_pred),2)/torch.linalg.norm(vt,2)        # Relative L2 Norm of the error (Vector)
        
        v_pred = np.reshape(v_pred.cpu().detach().numpy(),(256,256,256),order='F') 
        
        return error_vec, v_pred

# WAN0

In [6]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
    
        'Initialise neural network as a nn.MSELosslist using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            
            # weights from a normal distribution with 
            # Recommended gain value for tanh = 5/3?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            

    def forward(self,x):
        
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            
            z = self.linears[i](a)
                        
            a = self.activation(z)
            
        a = self.linears[-1](a)
        
        return a
                        
    def loss_BC(self,x,y):
                
        loss_v = self.loss_function(self.forward(x), y)
                
        return loss_v
    
    def loss_PDE(self, x_to_train_f):
                
        x_1_f = x_to_train_f[:,[0]]
        x_2_f = x_to_train_f[:,[1]]
        x_3_f = x_to_train_f[:,[2]]
        
        g = x_to_train_f.clone()
                        
        g.requires_grad_(True)
        
        vv = self.forward(g)
                
        vv_x = autograd.grad(vv,g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
                                                            
        vv_x_1 = vv_x[:,[0]]
        
        vv_x_2 = vv_x[:,[1]]
        
        vv_x_3 = vv_x[:,[2]]
                        
        k = x_1_f**2 + x_2_f**2 + x_3_f**2 + 1

        gg = (x_1_f**2 + x_2_f**2 + x_3_f**2 + 1)*(x_1_f**2 + x_2_f**2 +1)*np.sin(x_1_f*x_2_f+x_3_f)-(4*x_1_f*x_2_f+2*x_3_f)*np.cos(x_1_f*x_2_f+x_3_f)+1/(2*np.pi*np.sqrt(x_1_f**2 + x_2_f**2 + x_3_f**2))
        f = torch.log(torch.square(torch.mean(k * (u_x_1*vv_x_1 + u_x_2*vv_x_2+ u_x_3*vv_x_3))- 1e5-torch.mean(gg*u)))-torch.log(torch.mean(torch.square(u)))
        
        return f
    
    def loss(self,x,y,x_to_train_f):

        loss_v = self.loss_BC(x,y)
        loss_f = self.loss_PDE(x_to_train_f)

        loss = 1153/2 * loss_v + loss_f

        return loss
     
    'callable for optimizer'                                       
    def closure(self):
        
        optimizer.zero_grad()
        
        loss_val = self.loss(X_v_train, v_train, X_f_train)
        
        #error_vec, _ = WAN0.test()
        
        #print(loss_val,error_vec)
        
        global ite, err_vec, start_time
        ite = ite + 1
        
        if (ite % 100 == 0):
            time_vec.append(time.time() - start_time)
            error_vec, _ = WAN0.test()
            err_vec.append(error_vec.detach().float().item())
        
        loss_val.backward(retain_graph=True)

        return loss_val        
    
    def test(self):
                
        v_pred = self.forward(X_v_test_tensor)
        
        error_vec = torch.linalg.norm((vt-v_pred),2)/torch.linalg.norm(vt,2)        # Relative L2 Norm of the error (Vector)
        
        v_pred = np.reshape(v_pred.cpu().detach().numpy(),(256,256,256),order='F') 
        
        return error_vec, v_pred

# Loss Function

In [7]:
N_v = 600 
N_f = 10000 

X_f_train_np_array, X_v_train_np_array, v_train_np_array = trainingdata(N_v,N_f)

'Convert to tensor and send to GPU'
X_f_train = torch.from_numpy(X_f_train_np_array).float().to(device)
X_v_train = torch.from_numpy(X_v_train_np_array).float().to(device)
v_train = torch.from_numpy(v_train_np_array).float().to(device)
X_v_test_tensor = torch.from_numpy(X_v_test).float().to(device)
vt = torch.from_numpy(v_true).float().to(device)

layers = np.array([3, 5, 5, 1])
layers0 = np.array([3, 5, 5, 1])

WAN0 = Sequentialmodel(layers)
WAN0.to(device)

time_vec = []
err_vec = []

start_time = time.time()

x_1_f = X_f_train[:,[0]]
x_2_f = X_f_train[:,[1]]
x_3_f = X_f_train[:,[2]]
g = X_f_train.clone()                       
g.requires_grad = True      
v = WAN0.forward(g)              
v_x = autograd.grad(v,g,torch.ones([X_f_train.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
v_x_1 = v_x[:,[0]]
v_x_2 = v_x[:,[1]]
v_x_3 = v_x[:,[2]]

WAN = Sequentialmodel0(layers0)
WAN.to(device)

optimizer = torch.optim.LBFGS(WAN.parameters(), lr=0.11, 
                              max_iter = 1000, 
                              max_eval = 2500, 
                              tolerance_grad = 1e-06, 
                              tolerance_change = 1e-09, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')

optimizer.zero_grad()     # zeroes the gradient buffers of all parameters
optimizer.step(WAN.closure)

x_1_f = X_f_train[:,[0]]
x_2_f = X_f_train[:,[1]]
x_3_f = X_f_train[:,[2]]
g = X_f_train.clone()                       
g.requires_grad = True      
u = WAN.forward(g)              
u_x = autograd.grad(u,g,torch.ones([X_f_train.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]                                                          
u_x_1 = u_x[:,[0]]      
u_x_2 = u_x[:,[1]]
u_x_3 = u_x[:,[2]]

PINN = Sequentialmodel(layers)

ite = 0
optimizer = torch.optim.LBFGS(WAN0.parameters(), lr=0.11, 
                              max_iter = 5000, 
                              max_eval = None, 
                              tolerance_grad = 1e-06, 
                              tolerance_change = 1e-09, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')

optimizer.zero_grad()     # zeroes the gradient buffers of all parameters
optimizer.step(WAN0.closure)

elapsed = time.time() - start_time                
print('Training time: %.2f' % (elapsed))


''' Model Accuracy ''' 
error_vec, v_pred = WAN0.test()

print('Test Error: %.5f'  % (error_vec))
print(time_vec)
print(err_vec)

Training time: 57.02
Test Error: 0.06563
[4.189789295196533, 10.732282876968384, 14.179094076156616, 17.638709545135498, 20.74244499206543, 23.621702671051025, 26.552860021591187, 29.290534496307373, 32.04020643234253, 34.91648030281067, 37.92842173576355, 40.790764808654785, 43.6441285610199, 46.36587834358215, 49.09358215332031, 51.73647713661194, 54.378440856933594]
[0.4417053163051605, 0.3198179304599762, 0.18629775941371918, 0.19712841510772705, 0.1873161941766739, 0.1933523714542389, 0.199367493391037, 0.2032577097415924, 0.19613927602767944, 0.16525226831436157, 0.16073870658874512, 0.16797934472560883, 0.16112980246543884, 0.11776256561279297, 0.07108234614133835, 0.06725131720304489, 0.06520531326532364]
